In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import pandas as pd

from rail_coder.models import Document, DocumentCode
documents = Document.objects.all()

In [2]:
documents = Document.objects.all()

In [3]:
d = documents[0]

In [5]:
d.documentcode_set.latest('date').uses_restricted

'discrimination, exploitation, legally protected characteristics, legally protected vulnerabilities'

In [18]:
# for every document get its most recent document code
data = []

for doc in documents:
    d = {"id": doc.id,
            "clause": doc.clause_text,
            "domain": doc.domain,
            "tags": doc.tags,
            "source": doc.source}

    # most recent DocumentCode
    try:
        dcode = doc.documentcode_set.latest('date')
    except DocumentCode.DoesNotExist as dne:
        print(doc)
        raise dne

    d['coded_date'] = dcode.date
    d['artifacts_restricted'] = [x.strip() for x in dcode.artifacts_restricted.split(",")]
    d['uses_restricted'] = [x.strip() for x in dcode.uses_restricted.split(",")]
    d['how_restricted'] = [x.strip() for x in dcode.how_restricted.split(",")]
    data.append(d)
    

df = pd.DataFrame(data).set_index("id")

In [19]:
df.head()

,clause,domain,tags,source,coded_date,artifacts_restricted,uses_restricted,how_restricted
id,,,,,,,,
1,Licensed artifact(s) shall not discriminate or...,Discrimination,"discrimination, legally protected characterist...",RAIL (IEEE),2023-10-11 22:14:52.041367+00:00,[licensed artifacts],"[discrimination, exploitation, legally protect...",[prohibition]
2,Licensed artifact(s) shall not be used for wea...,Military,"military, weapons",RAIL (IEEE),2023-10-11 22:15:14.824527+00:00,[licensed artifacts],"[weaponry, warfare]",[prohibition]
3,Licensed artifact(s) shall not engage or enabl...,Legal,"automated decision-making, legal rights",RAIL (IEEE),2023-10-11 22:22:30.932441+00:00,[licensed artifacts],"[fully automated decision-making, adverse impa...",[prohibition]
4,Licensed artifact(s) shall not engage or enabl...,Legal,"automated decision-making, legal bindings",RAIL (IEEE),2023-10-12 20:20:50.817788+00:00,[licensed artifacts],"[automated decision making, enforceable obliga...",[]
5,Licensed artifact(s) shall not predict a natur...,Discrimination,"crime forcasting, prediction",RAIL (IEEE),2023-10-11 22:25:19.541248+00:00,[licensed artifacts],[crime prediction],[prohibition]


# Use Restriction Tags

In [20]:
df.explode('uses_restricted').uses_restricted.value_counts()

uses_restricted
misinformation                                   9
violation of the law                             8
disinformation                                   8
behavioral manipulation                          7
violation of law                                 7
                                                ..
International Traffic Arms Regulations (ITAR)    1
purpose of harming others                        1
publicly accessible space                        1
user identification                              1
disclosure of known dangers of AI system         1
Name: count, Length: 119, dtype: int64

# Vulnerabilities

In [24]:
for ind, row in df[df['clause'].str.contains("vulnerab")].iterrows():
    print(row['source'])
    print("\t"+row['clause'])
    print("\n")

RAIL (IEEE)
	Licensed artifact(s) shall not discriminate or exploit individuals or groups based on legally protected characteristics and/or vulnerabilities; and neither be deployed or used for those purposes.


RAIL (IEEE)
	Licensed artifact(s) shall not utilize personal information to infer additional personal information about a natural person, including but not limited to legally protected characteristics, vulnerabilities or categories; unless informed consent from the data subject to collect said inferred personal information for a stated purpose and defined duration is received.


AI Pubs
	You agree not to use the <licensed artifact(s)> or its Derivatives to exploit any of the vulnerabilities of a specific group of persons based on their age, social, physical or mental characteristics, in order to materially distort the behavior of a person belonging to that group in a manner that causes or is likely to cause that person or another person Harm.


Big Code
	You agree not to Use the